In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [15]:
# Hyper parameters
num_epochs = 2
output_classes = 10
batch_size = 20
learning_rate = 0.001

In [16]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [17]:
# Convolutional Neural Network
class CNN(nn.Module):
    def __init__(self, output_classes):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # 每次做maxpooling就把原图片减小了一倍的大小
        self.linear = nn.Linear(7*7*32, output_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1) # 将最后的整张图片展开
        out = self.linear(out)
        return out

In [18]:
model = CNN(output_classes)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [20]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/3000], Loss: 0.1431
Epoch [1/2], Step [200/3000], Loss: 0.0958
Epoch [1/2], Step [300/3000], Loss: 0.1180
Epoch [1/2], Step [400/3000], Loss: 0.2312
Epoch [1/2], Step [500/3000], Loss: 0.3715
Epoch [1/2], Step [600/3000], Loss: 0.0059
Epoch [1/2], Step [700/3000], Loss: 0.0479
Epoch [1/2], Step [800/3000], Loss: 0.1921
Epoch [1/2], Step [900/3000], Loss: 0.0834
Epoch [1/2], Step [1000/3000], Loss: 0.0994
Epoch [1/2], Step [1100/3000], Loss: 0.0295
Epoch [1/2], Step [1200/3000], Loss: 0.1361
Epoch [1/2], Step [1300/3000], Loss: 0.2394
Epoch [1/2], Step [1400/3000], Loss: 0.0417
Epoch [1/2], Step [1500/3000], Loss: 0.0164
Epoch [1/2], Step [1600/3000], Loss: 0.0054
Epoch [1/2], Step [1700/3000], Loss: 0.1106
Epoch [1/2], Step [1800/3000], Loss: 0.2651
Epoch [1/2], Step [1900/3000], Loss: 0.0245
Epoch [1/2], Step [2000/3000], Loss: 0.0101
Epoch [1/2], Step [2100/3000], Loss: 0.1104
Epoch [1/2], Step [2200/3000], Loss: 0.2344
Epoch [1/2], Step [2300/3000], Loss: 0.11

In [21]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 98.34 %
